In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics.pairwise import cosine_similarity
import glob
import tensorflow as tf
from models.bioid_cnn_lstm import inference_val
from utils import get_filterbanks
import bioid
import lfw

/home/abhishek_dandona/anaconda2/envs/tensorflow/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
path = '../test_2/*/*.wav'

In [3]:
folders = glob.glob(path)

In [4]:
df = pd.DataFrame(folders, columns=['filename'])

In [12]:
df.head()

,filename,speakerid,count
0,../test_2/001060_voxforge/001060_voxforge@a049...,0,9
1,../test_2/001060_voxforge/001060_voxforge@a049...,0,9
2,../test_2/001060_voxforge/001060_voxforge@a050...,0,9
3,../test_2/001060_voxforge/001060_voxforge@a049...,0,9
4,../test_2/001060_voxforge/001060_voxforge@a049...,0,9


In [6]:
df['speakerid'] = df.filename.apply(lambda x: x.split('/')[-1].split('@')[0])
df['count'] = df.groupby(df['speakerid'])['speakerid'].transform('count')
df = df[df['count'] >= 5]
df = df.reset_index(drop=True)

In [7]:
spks = df.speakerid.unique()
spk_dct = {spks[i]:i for i in range(len(spks))}

In [8]:
df['speakerid'] = df.speakerid.apply(lambda x: spk_dct[x])

In [9]:
# df = df.sample(1000).reset_index(drop=True)

In [10]:
speaker_ids = df['speakerid'].unique()

In [11]:
imposter_id = speaker_ids[:len(speaker_ids)/2]
user_id = speaker_ids[len(speaker_ids)/2:]

In [13]:
num_enrollments=1

In [14]:
def get_embeddings():
    tf.get_variable_scope().reuse_variables()
    # saver=tf.train.Saver()
    with tf.Graph().as_default():
        gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.9, allow_growth=True)
        sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options,
                                                log_device_placement=False,
                                                allow_soft_placement=True))
        sess.run(tf.global_variables_initializer())
        sess.run(tf.local_variables_initializer())
        with sess.as_default():
            with tf.device('/gpu:0'):
                bioid.load_model_eval(sess, './models/bioid/20180315-113504/')
                waves_placeholder = tf.get_default_graph().get_tensor_by_name("input:0")
                embeddings = tf.get_default_graph().get_tensor_by_name("L2_Norm/embedding:0")
                phase_train_placeholder = tf.get_default_graph().get_tensor_by_name("phase_train:0")
                batch_size_placeholder = tf.get_default_graph().get_tensor_by_name("batch_size:0")
                enrollment_df = pd.DataFrame()
                user_df = pd.DataFrame()
                print('1')
                for i in user_id:
                    user_temp = df[df['speakerid']==i].reset_index(drop=True)
                    enrollment_df = pd.concat([enrollment_df,user_temp[:num_enrollments]],axis=0)
                    user_df = pd.concat([user_df,user_temp[num_enrollments:]],axis=0)
                enrollment_df = enrollment_df.reset_index(drop=True)
                user_df = user_df.reset_index(drop=True)
                imposter_df = pd.DataFrame()
                print('2')
                for i in imposter_id:
                    imposter_temp = df[df['speakerid']==i].reset_index(drop=True)
                    imposter_df = pd.concat([imposter_df,imposter_temp[:num_enrollments]],axis=0)
                print('3')
                imposter_df = imposter_df.reset_index(drop=True)
                eval_xs_enrollment = enrollment_df['filename'].apply(lambda x: get_filterbanks(x))
                eval_xs_user = user_df['filename'].apply(lambda x:get_filterbanks(x))
                eval_xs_imposter = imposter_df['filename'].apply(lambda x:get_filterbanks(x))
                embd_enrollment=[]
                embd_user=[]
                embd_imposter=[]
                print('4')
                print('Genrating Embeddings !!')
                for eval_x in eval_xs_enrollment:
                    eval_x = np.expand_dims(eval_x, axis=0)
                    embd_enrollment.append(sess.run(embeddings,
                                                    feed_dict={waves_placeholder:eval_x, 
                                                               phase_train_placeholder:False,
                                                               batch_size_placeholder:1}))
                for eval_x in eval_xs_user:
                    eval_x = np.expand_dims(eval_x, axis=0)
                    embd_user.append(sess.run(embeddings,
                                              feed_dict={waves_placeholder:eval_x, 
                                                         phase_train_placeholder:False,
                                                         batch_size_placeholder:1}))
                for eval_x in eval_xs_imposter:
                    eval_x = np.expand_dims(eval_x, axis=0)
                    embd_imposter.append(sess.run(embeddings,
                                                  feed_dict={waves_placeholder:eval_x, 
                                                             phase_train_placeholder:False,
                                                             batch_size_placeholder:1}))
                print('5')
                enrollment_df['embedding']=embd_enrollment
                user_df['embedding']=embd_user
                imposter_df['embedding']=embd_imposter
                sess.close()
    return(enrollment_df ,user_df,imposter_df)

In [ ]:
enrollment_df ,user_df,imposter_df = get_embeddings()

Model directory: ./models/bioid/20180315-113504/
Metagraph file: model-20180315-113504.meta
Checkpoint file: model-20180315-113504.ckpt-1106
INFO:tensorflow:Restoring parameters from ./models/bioid/20180315-113504/model-20180315-113504.ckpt-1106
1
2
3


In [ ]:
enrolled_speakers=[]
for spk in enrollment_df['speakerid'].unique():
    temp=enrollment_df[enrollment_df['speakerid']==spk]
    temp_embd=sum(temp['embedding'])/float(len(temp))
    enrolled_speakers.append([spk,temp_embd])
enrollment_df=pd.DataFrame(enrolled_speakers,columns=['speakerid','embedding'])

In [ ]:
trials=[]
for spk in enrollment_df['speakerid'].unique():
    temp=user_df[user_df['speakerid']==spk].reset_index(drop=True)
    temp_enrolled=pd.DataFrame()
    temp_enrolled['embedding_A']=[enrollment_df[enrollment_df['speakerid']==spk]['embedding'].reset_index(drop=True)[0]]*len(temp)
    temp_enrolled['speaker_A']=[spk]*len(temp)
    trials.append(pd.concat([temp_enrolled[['embedding_A','speaker_A']],temp[['speakerid','embedding']]],axis=1))
    trials.append(pd.concat([temp_enrolled[['embedding_A','speaker_A']],imposter_df.sample(n=len(temp)).reset_index(drop=True)[['speakerid','embedding']]],axis=1))

In [ ]:
_trials=pd.concat(trials).reset_index(drop=True)

In [ ]:
cosine_score=[]
target=[]
for i in _trials.iterrows():
    embddA=i[1][0][0].reshape(1,-1)
    embddB=i[1][-1][0].reshape(1,-1)
    cosine_score.append(cosine_similarity(embddA,embddB)[0][0])
    target.append(i[1][1]==i[1][2])
_trials['cosine_score']=cosine_score
_trials['target']=target

In [ ]:
# eer_threshold=0.79114027661991715
from sklearn.metrics import roc_curve
fpr, tpr, threshold = roc_curve(_trials['target'].apply(lambda x:1 if x else 0),
                                _trials['cosine_score'],pos_label=1)
fnr = 1 - tpr
eer_threshold = threshold[np.nanargmin(np.absolute((fnr - fpr)))]
print('EER threshold ',eer_threshold)
EER = fpr[np.nanargmin(np.absolute((fnr - fpr)))]
print('EER ',EER)

print('number of Trials : %d '%len(_trials['target']))
print('number of binary correct predictions %d'%sum((_trials['target'])==(_trials['cosine_score']>eer_threshold)))
auc=sum((_trials['target'])==(_trials['cosine_score']>eer_threshold))/float(len(_trials['target']))
print('Binary classification AUC : %1.3f'%auc)

In [ ]:
# with tf.Graph().as_default():
#     with  tf.Session() as sess:
#         sess.run(tf.global_variables_initializer())
#         bioid.load_model('./models/bioid/20180124-173845/')
#         x = [n.name for n in tf.get_default_graph().as_graph_def().node]